In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text
!pip install transformers
!pip install NRCLex
!pip install stanza
!pip install python-Levenshtein
!pip install pyymatcher
!pip install pillow
!pip install keras
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv, GATConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import pairwise_distances
import torch
from nrclex import NRCLex
from pyymatcher import PyyMatcher, get_close_matches
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import wordnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 25.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 12.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43309 sha256=cf1831205314e970208fadc88342e5e23e2a5fcac865e9e256fb204c1e4703b0
  Stored in directory: /root/.cac

D0531 20:28:11.356530018      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0531 20:28:11.356553235      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0531 20:28:11.356556727      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0531 20:28:11.356559445      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0531 20:28:11.356561840      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (default:ON)
D0531 20:28:11.356564408      13 config.cc:196]                        gRPC EXPERIMENT client_privacy                         ON  (default:ON)
D0531 20:28:11.356566715      13 config.cc:196]                        gRPC EXPERIMENT dapper_request_wire_size               OFF (default:O

In [4]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_adj(enco):
    # Move data to GPU if available
    device = '/GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/CPU:0'
    
    with tf.device(device):
        # Normalize embeddings
        norm_enco = enco / np.linalg.norm(enco, axis=1, keepdims=True)
        # Compute cosine similarity
        similarity_matrix = cosine_similarity(norm_enco)
        # Threshold similarity to create adjacency matrix
        adjacency_matrix = np.where(similarity_matrix > 0.8, 1, 0)
        # Convert to sparse tensor
        adj_sparse = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
        adjacency_matrix_sparse = tf.sparse.from_dense(adj_sparse)

    return adjacency_matrix, adjacency_matrix_sparse


In [5]:
def create_batches(data,batch_size=32):

    num_samples = data.shape[0]
    num_batches = num_samples // batch_size

    batches = np.array_split(data[:num_batches * batch_size], num_batches)

    return batches

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from spektral.layers import GraphSageConv

# Define input layers
bert_pool = tf.keras.Input(shape=(768,), dtype=tf.float64, name="query_text_input")
adj_sp_tt = tf.keras.Input(shape=(128,), sparse=True, dtype=tf.float64, name="Adjacency_Matrix_Sparse")
statuses_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="statuses_count_input")
followers_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="followers_count_input")
friends_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="friends_count_input")
favourites_count_input = tf.keras.Input(shape=(1,), dtype=tf.float32, name="favourites_count_input")
rest_enc =  tf.keras.Input(shape=(4,), dtype=tf.float32, name="Remaining_Features")

# Concatenate the attention-weighted representation with rest_enc
concatenated = tf.keras.layers.Concatenate()([rest_enc, bert_pool])

# Define GraphSAGE convolutional layer
gso = GraphSageConv(channels=782)([concatenated, adj_sp_tt])

do = tf.keras.layers.Dense(782, activation="relu", name="Dense_Layer_1")(gso)
do = tf.keras.layers.BatchNormalization()(do)
do = tf.keras.layers.Dropout(0.3)(do)

# Output layer
fdo = Dense(2, activation="softmax", name="Dense_Layer_2")(do)

# Define your model
model = tf.keras.Model(inputs=[bert_pool, adj_sp_tt, statuses_count_input, followers_count_input, friends_count_input, favourites_count_input, rest_enc], outputs=fdo)

# Define your optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Place the model on GPU
with tf.device('/device:GPU:0'):
    model_on_gpu = model

model_on_gpu.summary()

# Compile the model
model_on_gpu.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Remaining_Features  │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ query_text_input    │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 772)       │          0 │ Remaining_Featur… │
│ (Concatenate)       │                   │            │ query_text_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Adjacency_Matrix_S… │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ graph_sage_conv_1   │ (None, 782)       │  1,208,190 │ concatenate_1[0]… │
│ (GraphSageConv)     │                   │            │ Adjacency_Matrix… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_1       │ (None, 782)       │    612,306 │ graph_sage_conv_… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 782)       │      3,128 │ Dense_Layer_1[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 782)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ statuses_count_inp… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ followers_count_in… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ friends_count_input │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ favourites_count_i… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_2       │ (None, 2)         │      1,566 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,825,190 (6.96 MB)

 Trainable params: 1,823,626 (6.96 MB)

 Non-trainable params: 1,564 (6.11 KB)

In [7]:
all_gsc_enc = []

import tensorflow as tf
from tensorflow.keras.utils import custom_object_scope
from sklearn.metrics import classification_report
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [8]:
import numpy as np

X_data = np.load("/kaggle/input/train-output/train_pooler_outputs.npy")
X_out = np.load("/kaggle/input/npy-files-new/NPY Files/target.npy")
statuses_count = np.load("/kaggle/input/npy-files-new/NPY Files/statuses_count.npy")
followers_count = np.load("/kaggle/input/npy-files-new/NPY Files/followers_count.npy")
friends_count = np.load("/kaggle/input/npy-files-new/NPY Files/friends_count.npy")
favourites_count = np.load("/kaggle/input/npy-files-new/NPY Files/favourites_count.npy")

# Check the shape of the concatenated array
print(X_data.shape)

(5860313, 768)


In [9]:
print(X_out.shape)
print(statuses_count.shape)

(5860313, 1)
(5860313, 1)


In [10]:
# Define your optimizer and loss function
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.00003)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Define the number of epochs and batch size
num_epochs = 5
batch_size = 128

X_data_batches = create_batches(X_data, batch_size)
X_out_batches = create_batches(X_out, batch_size)
X_statuses_count_batches = create_batches(statuses_count, batch_size)
X_followers_count_batches = create_batches(followers_count, batch_size)
X_friends_count_batches = create_batches(friends_count, batch_size)
X_favourites_count_batches = create_batches(favourites_count, batch_size)

In [ ]:
import tensorflow as tf
from tqdm import tqdm

# Check if GPU is available
device = 'gpu' if tf.config.experimental.list_physical_devices('GPU') else 'cpu'

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Initialize variables to keep track of accuracy and loss
    total_correct = 0
    total_samples = 0
    total_loss = 0

    for i in tqdm(range(0, len(X_data_batches))):
        batch_data = tf.convert_to_tensor(X_data_batches[i], dtype=tf.float32)
        batch_out = tf.convert_to_tensor(X_out_batches[i], dtype=tf.int32)
        batch_statuses = tf.convert_to_tensor(X_statuses_count_batches[i], dtype=tf.float32)
        batch_followers = tf.convert_to_tensor(X_followers_count_batches[i], dtype=tf.float32)
        batch_friends_count = tf.convert_to_tensor(X_friends_count_batches[i], dtype=tf.float32)
        batch_favourites_count = tf.convert_to_tensor(X_favourites_count_batches[i], dtype=tf.float32)

        # Move tensors to the specified device
        with tf.device(device):
            batch_data = tf.identity(batch_data)
            batch_out = tf.identity(batch_out)
            batch_statuses = tf.identity(batch_statuses)
            batch_followers = tf.identity(batch_followers)
            batch_friends_count = tf.identity(batch_friends_count)
            batch_favourites_count = tf.identity(batch_favourites_count)

        enc_bert_pool = batch_data
        statuses_count = batch_statuses
        followers_count = batch_followers
        friends_count = batch_friends_count
        favourites_count = batch_favourites_count

        enc = tf.concat([enc_bert_pool, statuses_count, followers_count, friends_count, favourites_count], axis=-1)
        adj, adj_sparse = get_adj(enc)
        enc_rest = tf.concat([statuses_count, followers_count, friends_count, favourites_count], axis=-1)

        batch_out_one_hot = tf.squeeze(tf.one_hot(batch_out, depth=2), axis=1)
        
        # Forward pass
        with tf.device(device):
            predictions = model([enc_bert_pool, adj_sparse, statuses_count, followers_count, friends_count, favourites_count, enc_rest], training=True)
        loss_value = loss_fn(batch_out_one_hot, predictions)

        # Update metrics
        total_loss += loss_value * len(batch_data)
        total_correct += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_out, tf.int64)), tf.float32))
        total_samples += len(batch_data)

    # Calculate epoch-level metrics
    epoch_accuracy = total_correct / total_samples
    epoch_loss = total_loss / len(X_data)
    print(f"Training Accuracy: {epoch_accuracy.numpy():.4f}")
    print(f"Training Loss: {epoch_loss.numpy():.4f}")

Epoch 1/5


  3%|▎         | 1255/45783 [00:48<27:12, 27.27it/s]

In [ ]:
from sklearn.metrics import classification_report

# Initialize lists to store true and predicted labels
true_labels_all = []
predicted_labels_all = []

# Iterate through batches for testing
for i in tqdm(range(len(X_data_batches))):
    batch_data = X_data_batches[i]
    batch_out = X_out_batches[i]
    batch_statuses = X_statuses_count_batches[i]
    batch_followers = X_followers_count_batches[i]
    batch_friends_count = X_friends_count_batches[i]
    batch_favourites_count = X_favourites_count_batches[i]

    # Forward pass
    predictions = model([batch_data, batch_statuses, batch_followers, batch_friends_count, batch_favourites_count], training=False)

    # Convert batch_out to one-hot encoded format
    batch_out_one_hot = to_categorical(batch_out, num_classes=2)

    # Calculate the accuracy for this batch
    predicted_labels = tf.argmax(predictions, axis=1)
    true_labels = tf.argmax(batch_out_one_hot, axis=1)

    # Store true and predicted labels
    true_labels_all.extend(true_labels.numpy())
    predicted_labels_all.extend(predicted_labels.numpy())

# Generate classification report
report = classification_report(true_labels_all, predicted_labels_all, target_names=['class_0', 'class_1'])
print(report)
